# Dataset 

[dataset link](https://www.kaggle.com/paultimothymooney/blood-cells)

Goal = Classify  types of WBCs

Task 1:
- Download Data
- Unzip it
- split them
- save result in a correct way
 - User_Path + dataset/train
 - User_path + dataset/test

# Help

- Kaggle user and key
- dataset path destination 
- test size split

# Requirments

In [1]:
! pip install kaggle
! pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=57d64f8d85dd583a2a52055f249cf1e7079bc69533b8056b19aef1c3429cebea
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


# Libraries

In [2]:
# import kaggle
import wget
import os
import zipfile
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

# Config


In [3]:
kaggle_user = "monibaravan"
kaggle_key = "7a03bfd58536ced3a30c3b5c742096dd"
des_path = "" # destination path
test_size = 0.3

# Loading Data


## download from Kaggle

Output: zip file

In [4]:
os.environ['KAGGLE_USERNAME'] = kaggle_user # username from the json file
os.environ['KAGGLE_KEY'] = kaggle_key # key from the json file

In [5]:
!kaggle datasets download -d paultimothymooney/blood-cells

 97% 105M/108M [00:02<00:00, 41.8MB/s] 
100% 108M/108M [00:02<00:00, 41.7MB/s]


In [6]:
dataset_path = "blood-cells.zip"
data_path = "dataset-master/dataset-master/JPEGImages"
label_csv_path = "dataset-master/dataset-master/labels.csv"

## Loading
Input: zip file

Output: 

dataset/train

dataset/test


In [7]:
zipfile.ZipFile(dataset_path).extractall()

In [8]:
# labels in .csv
labels_df = pd.read_csv(label_csv_path)
# labels_df.head()

In [9]:
labels_df = labels_df.dropna(subset=['Image', 'Category']) 
labels_df['Image'] = labels_df['Image'].apply(
    lambda x: 'BloodImage_0000' + str(x) + '.jpg' 
    if x < 10 
    else ('BloodImage_00' + str(x) + '.jpg' if x > 99 else 'BloodImage_000' + str(x) + '.jpg')
)
labels_df = labels_df[['Image', 'Category']]
# labels_df.head()

In [10]:
cat_counts = labels_df.Category.value_counts()
to_remove = cat_counts[cat_counts < 10].index 
labels_df = labels_df.replace(to_remove, np.nan).dropna()
# labels_df.Category.value_counts()

In [11]:
classes = labels_df.Category.unique()
idx_classes = {}
for i, cls in enumerate(classes):
  idx_classes[ cls ] = i

# print(idx_classes)

# Split Train and Test

In [12]:
train_df, test_df = train_test_split(
    labels_df, 
    test_size = test_size,
    stratify = labels_df['Category']
)
# print('shape of data split: ', 'train:', train_df.shape, 'test:', test_df.shape)

# Saving Data

In [13]:
# !rm -r dataset

In [14]:
os.makedirs(os.path.join(des_path, 'dataset'))
os.makedirs(os.path.join(des_path,'dataset', 'train'))
# os.makedirs(os.path.join(des_path,'dataset', 'validation'))
os.makedirs(os.path.join(des_path,'dataset', 'test'))

In [15]:
for cls in classes:
  os.makedirs(os.path.join(des_path,'dataset', 'train', cls))
  # os.makedirs(os.path.join(des_path,'dataset', 'validation', cls))
  os.makedirs(os.path.join(des_path,'dataset', 'test', cls))

In [16]:
# TRAIN DATA
for index, row in train_df.iterrows():
  load_path = os.path.join(data_path, row['Image'])
  if not os.path.exists(load_path):
    continue

  img = cv2.imread(load_path) # Load each Image
  cls = row['Category'].strip() # Label
  save_path = os.path.join(des_path,'dataset', 'train', cls,  row['Image']) # save path
  # print(row['Image'], cls)
  cv2.imwrite(save_path, img) # save each image to its Category

In [17]:
# Test DATA
for index, row in test_df.iterrows():
  load_path = os.path.join(data_path, row['Image'])
  if not os.path.exists(load_path):
    continue

  img = cv2.imread(load_path) # Load each Image
  cls = row['Category'].strip() # Label
  save_path = os.path.join(des_path,'dataset', 'test', cls,  row['Image']) # save path
  # print(row['Image'], cls)
  cv2.imwrite(save_path, img) # save each image to its Category

In [18]:
print("Preprocessing is done!")

Preprocessing is done!
